In [94]:
import numpy as np
import pandas as pd
import os

In [95]:
# Leemos los datos
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [96]:
# Ponemos que se vean todas las columnas de un df
pd.set_option("display.max_columns", None)

In [97]:
train.shape, test.shape

((3695, 50), (2464, 49))

In [98]:
# Separamos el target del train
target = train['recidivism_label']
train.drop(['recidivism_label', 'Id'], axis=1, inplace=True)

In [99]:
# Creamos un validation set (solo con los IDs, que son las filas) que usaremos para validar cualquier modelo que hagamos. 

In [100]:
from sklearn.model_selection import train_test_split

In [101]:
index_train, index_val, y_train, y_val = train_test_split(train.index, target, test_size=0.30, random_state=42)

In [102]:
train.head()

,compas_screening_date,sex,dob,age,age_cat,race,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_jail_in,c_jail_out,c_case_number,c_offense_date,c_arrest_date,c_days_from_compas,c_charge_degree,c_charge_desc,is_recid,r_case_number,r_charge_degree,r_days_from_arrest,r_offense_date,r_charge_desc,r_jail_in,r_jail_out,violent_recid,is_violent_recid,vr_case_number,vr_charge_degree,vr_offense_date,vr_charge_desc,type_of_assessment,decile_score.1,score_text,screening_date,v_type_of_assessment,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,doughnuts_comsumption
0,2014-06-25,Male,1987-10-08,28,25 - 45,African-American,0,2,0,0,5,-1.0,2014-06-24 06:22:18,2014-06-27 09:51:57,14004144CF10A,NaN,2014-06-25,0.0,F,arrest case no charge,1,14002721MM20A,(M1),280.0,2014-09-19,Possess Cannabis/20 Grams Or Less,2015-06-26,2015-10-22,NaN,0,NaN,NaN,NaN,NaN,Risk of Recidivism,2,Low,2014-06-25,Risk of Violence,2,Low,2014-06-25,2014-06-24,2014-06-27,5,2,86,0.458606
1,2013-01-07,Male,1981-03-26,35,25 - 45,Caucasian,0,3,0,0,2,0.0,2013-01-07 02:38:57,2013-01-07 09:00:03,13000389MM10A,2013-01-07,NaN,0.0,M,Battery,1,15007671MM10A,(M2),0.0,2015-07-18,Disorderly Intoxication,2015-07-18,2015-07-18,NaN,0,NaN,NaN,NaN,NaN,Risk of Recidivism,3,Low,2013-01-07,Risk of Violence,4,Low,2013-01-07,2015-07-18,2015-07-18,2,0,922,0.006780
2,2013-02-23,Male,1990-01-14,26,25 - 45,African-American,0,10,0,0,7,-1.0,2013-02-22 05:14:38,2013-02-24 02:35:53,13002747CF10A,2013-02-22,NaN,1.0,F,Grand Theft in the 3rd Degree,1,13023460TC30A,(M2),NaN,2013-03-03,Operating W/O Valid License,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,Risk of Recidivism,10,High,2013-02-23,Risk of Violence,8,High,2013-02-23,2013-02-22,2013-02-24,7,1,8,0.439042
3,2013-12-22,Female,1972-05-28,43,25 - 45,Caucasian,0,1,0,0,0,0.0,2013-12-22 02:32:05,2013-12-23 01:40:39,13017627CF10A,2013-12-21,NaN,1.0,F,Aggravated Battery / Pregnant,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,Risk of Recidivism,1,Low,2013-12-22,Risk of Violence,1,Low,2013-12-22,2013-12-22,2013-12-23,0,1,831,0.257482
4,2014-08-07,Female,1991-03-07,25,25 - 45,Caucasian,0,3,0,0,2,-1.0,2014-08-06 12:07:31,2014-08-07 08:49:26,14028826MU10A,2014-08-06,NaN,1.0,M,Unlaw Lic Use/Disply Of Others,1,15030357TC10A,(M2),NaN,2015-10-23,Driving License Suspended,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,Risk of Recidivism,3,Low,2014-08-07,Risk of Violence,3,Low,2014-08-07,2014-08-06,2014-08-07,2,0,442,1.146142


In [103]:
# Vemos si hay alguna columna constante y las eliminamos
Univalor=list()
for col in list(train):
    l=train[col].unique()
    print(len(l),'\t',train[col].dtypes,'\t',col)
    if len(l)<=1:
        Univalor.append(col)

656 	 object 	 compas_screening_date
2 	 object 	 sex
3159 	 object 	 dob
62 	 int64 	 age
3 	 object 	 age_cat
6 	 object 	 race
9 	 int64 	 juv_fel_count
10 	 int64 	 decile_score
9 	 int64 	 juv_misd_count
8 	 int64 	 juv_other_count
34 	 int64 	 priors_count
51 	 float64 	 days_b_screening_arrest
3695 	 object 	 c_jail_in
3689 	 object 	 c_jail_out
3695 	 object 	 c_case_number
673 	 object 	 c_offense_date
292 	 object 	 c_arrest_date
103 	 float64 	 c_days_from_compas
2 	 object 	 c_charge_degree
317 	 object 	 c_charge_desc
2 	 int64 	 is_recid
1803 	 object 	 r_case_number
10 	 object 	 r_charge_degree
120 	 float64 	 r_days_from_arrest
898 	 object 	 r_offense_date
256 	 object 	 r_charge_desc
718 	 object 	 r_jail_in
698 	 object 	 r_jail_out
1 	 float64 	 violent_recid
2 	 int64 	 is_violent_recid
415 	 object 	 vr_case_number
9 	 object 	 vr_charge_degree
345 	 object 	 vr_offense_date
64 	 object 	 vr_charge_desc
1 	 object 	 type_of_assessment
10 	 int64 	 decile_score.1


In [104]:
train.drop(Univalor, axis=1, inplace=True)
test.drop(Univalor, axis=1, inplace=True)

In [105]:
#Convert dates to datetime format
date_vars = ['compas_screening_date', 'dob', 'c_offense_date', 'c_arrest_date', 'r_offense_date', 'r_jail_in', 'r_jail_out',
            'screening_date', 'v_screening_date', 'in_custody', 'out_custody', 'c_jail_in', 'c_jail_out', 'vr_offense_date']

for var in date_vars:
    train[var] = pd.to_datetime(train[var])
    test[var] = pd.to_datetime(test[var])

In [106]:
# Vemos los formatos que contiene el df
train.dtypes.unique()

array([dtype('<M8[ns]'), dtype('O'), dtype('int64'), dtype('float64')],
      dtype=object)

In [107]:
# Convertimos los int64 a float64
for col in train.select_dtypes(include='int64').columns:
    train[col] = train[col].astype('float64')
    test[col] = test[col].astype('float64')

In [108]:
train.dtypes.unique()

array([dtype('<M8[ns]'), dtype('O'), dtype('float64')], dtype=object)

In [109]:
# Seleccionamos las variables categóricas y sustituimos los NANs por el string 'NS'
train_cat_variables = train.select_dtypes(include='O').columns

for col in train_cat_variables:
    train.loc[train[col].isna(),col]='NS'
    test.loc[test[col].isna(),col]='NS'

In [110]:
# Definimos una función para ver si dos columnas categóricas son iguales (o difieren solo por el nombre de los labels)
def compare_categorical_series(data, feature_1, feature_2):
    values_1, *_ = pd.factorize(data[feature_1])
    values_2, *_ = pd.factorize(data[feature_2])

    return np.all(values_1 == values_2)

In [111]:
n_same = 0
for i in range(0, len(train_cat_variables)):
    for j in range(i+1, len(train_cat_variables)):
        if compare_categorical_series(data=train, feature_1 = train_cat_variables[i], feature_2 = train_cat_variables[j]):
            print(train_cat_variables[i], train_cat_variables[j])
            n_same=n_same+1
            
if n_same ==0:
    print('No hi ha variables categòriques iguals')

No hi ha variables categòriques iguals


In [112]:
# Vemos si dos columnas con formato date coinciden
n_same = 0
train_date_variables = train.select_dtypes(include='<M8[ns]').columns
for i in range(0, len(train_date_variables)):
    for j in range(i+1, len(train_date_variables)):
        if train[train[train_date_variables[i]] == train[train_date_variables[j]]].shape[0] == train.shape[0]:
            print(train_date_variables[i], train_date_variables[j])
            n_same=n_same+1
            
if n_same ==0:
    print('No hi ha variables datetime iguals')

compas_screening_date screening_date
compas_screening_date v_screening_date
screening_date v_screening_date


In [113]:
# Eliminamos las redundantes
train.drop(['compas_screening_date', 'v_screening_date'], axis=1, inplace=True)
test.drop(['compas_screening_date', 'v_screening_date'], axis=1, inplace=True)
train_date_variables = train.select_dtypes(include='<M8[ns]').columns

In [114]:
# Estudiamos la correlación de las columnas numéricas

train_num_variables = train.select_dtypes(include='number').columns

correlation = train[train_num_variables].corr().abs()
    
# Select upper triangle of correlation matrix
upper = correlation.where(np.triu(np.ones(correlation.shape), k=1).astype(bool))

In [115]:
# Eliminamos aquellas que tengan correlación 1
to_drop = [column for column in upper.columns if any(upper[column] == 1)]
to_drop

['decile_score.1', 'priors_count.1']

In [116]:
# Antes vemos si tienen NAs en las mismas filas (ya que la función de correlación simplemente elimina las filas donde una de las
#dos variables sea NA)
train[np.isnan(train[to_drop[0]]) == np.isnan(train[to_drop[1]])].shape[0] ==train.shape[0]

True

In [117]:
train.drop(to_drop[0], axis=1, inplace=True)
test.drop(to_drop[0], axis=1, inplace=True)
train_num_variables = train.select_dtypes(include='number').columns

In [118]:
# We calculate all possible new variables from existing date variables

In [119]:
data_dates_train = train[train_date_variables]
data_dates_test = test[train_date_variables]
pd.options.mode.chained_assignment = None

for i in range(0, len(train_date_variables)):
    for j in range(i+1, len(train_date_variables)):
        new_col = train_date_variables[i] + '_' + train_date_variables[j]
        data_dates_train[new_col] = (data_dates_train[train_date_variables[i]] - 
                                     data_dates_train[train_date_variables[j]]).dt.days
        
        data_dates_test[new_col] = (data_dates_test[train_date_variables[i]] - 
                                     data_dates_test[train_date_variables[j]]).dt.days

data_dates_train.drop(train_date_variables, axis=1, inplace=True)
data_dates_test.drop(train_date_variables, axis=1, inplace=True)

In [120]:
#Next we will select the most important date variables building a catboost model

In [121]:
from catboost import CatBoostClassifier
from catboost import Pool

In [122]:
TS=0.1 # Validation size
esr=100 # Early stopping rounds (when validation does not improve in these rounds, stops)

predictoras=list(data_dates_train)[1:-1]

pool_tr=Pool(data_dates_train.iloc[index_train, :], y_train)
pool_test=Pool(data_dates_train.iloc[index_val, :], y_val)

In [123]:
model_catboost_val = CatBoostClassifier(
		  eval_metric='Accuracy',
		  iterations=10000, # Very high value, to find the optimum
		  od_type='Iter', # Overfitting detector set to "iterations"
		  verbose=25) # Shows train/test metric every "verbose" trees

In [124]:
# "Technical" parameters of the model:
params = {'objective': 'Logloss',
		  'learning_rate': 0.02, # learning rate, lower -> slower but better prediction
		  'depth': 6, # Depth of the trees (values betwwen 5 and 10, higher -> more overfitting)
		  'l2_leaf_reg': 15, # L2 regularization (between 3 and 20, higher -> less overfitting)
		  'rsm': 0.2, # % of features to consider in each split (lower -> faster and reduces overfitting)
		  'subsample': 0.5, # Sample rate for bagging (only if bootstrap_type is Poisson, Bernoulli or MVS)
		  'max_ctr_complexity': 3, # The maximum number of features that can be combined
		  'bootstrap_type': 'Bernoulli'} # Affects important aspects of choosing a split for a tree

model_catboost_val.set_params(**params)

In [125]:
print('\nCatboost Fit (Validation)...\n')
model_catboost_val.fit(X=pool_tr,
						 eval_set=pool_test,
						 early_stopping_rounds=esr)


Catboost Fit (Validation)...

0:	learn: 0.9876257	test: 0.9837692	best: 0.9837692 (0)	total: 9.11ms	remaining: 1m 31s
25:	learn: 0.9949729	test: 0.9972949	best: 0.9972949 (13)	total: 442ms	remaining: 2m 49s
50:	learn: 0.9953596	test: 0.9972949	best: 0.9972949 (13)	total: 893ms	remaining: 2m 54s
75:	learn: 0.9953596	test: 0.9963931	best: 0.9972949 (13)	total: 1.35s	remaining: 2m 55s
100:	learn: 0.9961330	test: 0.9963931	best: 0.9972949 (13)	total: 1.8s	remaining: 2m 56s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.9972948602
bestIteration = 13

Shrink model to first 14 iterations.


In [126]:
importances = model_catboost_val.feature_importances_

importances = pd.DataFrame([data_dates_train.columns, importances])
importances = importances.transpose()
importances.columns = ['Variables', 'Importance']
importances = importances.sort_values('Importance', ascending = False)

In [127]:
train.drop(train_date_variables, axis=1, inplace=True)

In [128]:
importances.head()

,Variables,Importance
50,r_offense_date_out_custody,45.039937
23,c_jail_out_r_offense_date,25.577667
48,r_offense_date_screening_date,10.875603
14,c_jail_in_r_offense_date,8.221248
59,r_jail_out_out_custody,0.79994


In [37]:
# We select those that have importance at all

In [131]:
most_importants = importances.loc[importances['Importance']>0.2, 'Variables']
most_importants.shape

(26,)

In [132]:
train = train.join(data_dates_train[most_importants])

In [133]:
# Now we will extract some features from the case id variables, a veure si sona la flauta

In [134]:
lenght_id_cases = train['c_case_number'].str.len()
lenght_id_cases[lenght_id_cases!=13]

# No NANs in c_case_number column

lenght_id_cases_r = train['r_case_number'].str.len()
len(lenght_id_cases_r[lenght_id_cases_r!=13])

1893

In [135]:
#1893 NAs
len(train[train['r_case_number']=='NS'])

1893

In [136]:
train.loc[train['r_case_number']=='NS', 'r_case_number'] = np.nan

In [137]:
train.head()

,sex,age,age_cat,race,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_case_number,c_days_from_compas,c_charge_degree,c_charge_desc,is_recid,r_case_number,r_charge_degree,r_days_from_arrest,r_charge_desc,is_violent_recid,vr_case_number,vr_charge_degree,vr_charge_desc,score_text,v_decile_score,v_score_text,priors_count.1,start,end,doughnuts_comsumption,r_offense_date_out_custody,c_jail_out_r_offense_date,r_offense_date_screening_date,c_jail_in_r_offense_date,r_jail_out_out_custody,r_offense_date_in_custody,c_jail_in_r_jail_out,c_offense_date_out_custody,dob_in_custody,c_jail_out_out_custody,c_jail_in_c_jail_out,dob_screening_date,dob_r_offense_date,c_jail_in_in_custody,c_arrest_date_screening_date,vr_offense_date_in_custody,c_jail_in_vr_offense_date,c_jail_in_out_custody,screening_date_in_custody,r_jail_in_screening_date,vr_offense_date_out_custody,c_arrest_date_out_custody,c_arrest_date_vr_offense_date,c_jail_out_c_offense_date,c_jail_out_r_jail_in,dob_out_custody
0,Male,28.0,25 - 45,African-American,0.0,2.0,0.0,0.0,5.0,-1.0,14004144CF10A,0.0,F,arrest case no charge,1.0,14002721MM20A,(M1),280.0,Possess Cannabis/20 Grams Or Less,0.0,NS,NS,NS,Low,2.0,Low,5.0,2.0,86.0,0.458606,84.0,-84.0,86.0,-87.0,482.0,87.0,-485.0,NaN,-9756,0,-4,-9757,-9843.0,0,0.0,NaN,NaN,-3,1,366.0,NaN,-2.0,NaN,NaN,-364.0,-9759
1,Male,35.0,25 - 45,Caucasian,0.0,3.0,0.0,0.0,2.0,0.0,13000389MM10A,0.0,M,Battery,1.0,15007671MM10A,(M2),0.0,Disorderly Intoxication,0.0,NS,NS,NS,Low,4.0,Low,2.0,0.0,922.0,0.006780,0.0,-922.0,922.0,-922.0,0.0,0.0,-922.0,-922.0,-12532,-922,-1,-11610,-12532.0,-922,NaN,NaN,NaN,-922,-922,922.0,NaN,NaN,NaN,0.0,-922.0,-12532
2,Male,26.0,25 - 45,African-American,0.0,10.0,0.0,0.0,7.0,-1.0,13002747CF10A,1.0,F,Grand Theft in the 3rd Degree,1.0,13023460TC30A,(M2),NaN,Operating W/O Valid License,0.0,NS,NS,NS,High,8.0,High,7.0,1.0,8.0,0.439042,7.0,-7.0,8.0,-9.0,NaN,9.0,NaN,-2.0,-8440,0,-2,-8441,-8449.0,0,NaN,NaN,NaN,-2,1,NaN,NaN,NaN,NaN,2.0,NaN,-8442
3,Female,43.0,25 - 45,Caucasian,0.0,1.0,0.0,0.0,0.0,0.0,13017627CF10A,1.0,F,Aggravated Battery / Pregnant,0.0,NaN,NS,NaN,NS,0.0,NS,NS,NS,Low,1.0,Low,0.0,1.0,831.0,0.257482,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.0,-15183,0,-1,-15183,NaN,0,NaN,NaN,NaN,-1,0,NaN,NaN,NaN,NaN,2.0,NaN,-15184
4,Female,25.0,25 - 45,Caucasian,0.0,3.0,0.0,0.0,2.0,-1.0,14028826MU10A,1.0,M,Unlaw Lic Use/Disply Of Others,1.0,15030357TC10A,(M2),NaN,Driving License Suspended,0.0,NS,NS,NS,Low,3.0,Low,2.0,0.0,442.0,1.146142,442.0,-442.0,442.0,-443.0,NaN,443.0,NaN,-1.0,-8553,0,-1,-8554,-8996.0,0,NaN,NaN,NaN,-1,1,NaN,NaN,NaN,NaN,1.0,NaN,-8554


In [138]:
case_id_vars = ['c_case_number', 'r_case_number']

In [139]:
for var in case_id_vars:

    train[var + 'last'] = train[var].str[-1:]
    train[var+'first'] = train[var].str[:2]
    train[var+'letters'] = train[var].str[8:10]
    
    new_vars = [var + 'last', var+'first', var+'letters']
    
    for new_var in new_vars:
            
        train.loc[train[new_var].isna(),new_var]='No_Idea'

In [140]:
train.head()

,sex,age,age_cat,race,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_case_number,c_days_from_compas,c_charge_degree,c_charge_desc,is_recid,r_case_number,r_charge_degree,r_days_from_arrest,r_charge_desc,is_violent_recid,vr_case_number,vr_charge_degree,vr_charge_desc,score_text,v_decile_score,v_score_text,priors_count.1,start,end,doughnuts_comsumption,r_offense_date_out_custody,c_jail_out_r_offense_date,r_offense_date_screening_date,c_jail_in_r_offense_date,r_jail_out_out_custody,r_offense_date_in_custody,c_jail_in_r_jail_out,c_offense_date_out_custody,dob_in_custody,c_jail_out_out_custody,c_jail_in_c_jail_out,dob_screening_date,dob_r_offense_date,c_jail_in_in_custody,c_arrest_date_screening_date,vr_offense_date_in_custody,c_jail_in_vr_offense_date,c_jail_in_out_custody,screening_date_in_custody,r_jail_in_screening_date,vr_offense_date_out_custody,c_arrest_date_out_custody,c_arrest_date_vr_offense_date,c_jail_out_c_offense_date,c_jail_out_r_jail_in,dob_out_custody,c_case_numberlast,c_case_numberfirst,c_case_numberletters,r_case_numberlast,r_case_numberfirst,r_case_numberletters
0,Male,28.0,25 - 45,African-American,0.0,2.0,0.0,0.0,5.0,-1.0,14004144CF10A,0.0,F,arrest case no charge,1.0,14002721MM20A,(M1),280.0,Possess Cannabis/20 Grams Or Less,0.0,NS,NS,NS,Low,2.0,Low,5.0,2.0,86.0,0.458606,84.0,-84.0,86.0,-87.0,482.0,87.0,-485.0,NaN,-9756,0,-4,-9757,-9843.0,0,0.0,NaN,NaN,-3,1,366.0,NaN,-2.0,NaN,NaN,-364.0,-9759,A,14,CF,A,14,MM
1,Male,35.0,25 - 45,Caucasian,0.0,3.0,0.0,0.0,2.0,0.0,13000389MM10A,0.0,M,Battery,1.0,15007671MM10A,(M2),0.0,Disorderly Intoxication,0.0,NS,NS,NS,Low,4.0,Low,2.0,0.0,922.0,0.006780,0.0,-922.0,922.0,-922.0,0.0,0.0,-922.0,-922.0,-12532,-922,-1,-11610,-12532.0,-922,NaN,NaN,NaN,-922,-922,922.0,NaN,NaN,NaN,0.0,-922.0,-12532,A,13,MM,A,15,MM
2,Male,26.0,25 - 45,African-American,0.0,10.0,0.0,0.0,7.0,-1.0,13002747CF10A,1.0,F,Grand Theft in the 3rd Degree,1.0,13023460TC30A,(M2),NaN,Operating W/O Valid License,0.0,NS,NS,NS,High,8.0,High,7.0,1.0,8.0,0.439042,7.0,-7.0,8.0,-9.0,NaN,9.0,NaN,-2.0,-8440,0,-2,-8441,-8449.0,0,NaN,NaN,NaN,-2,1,NaN,NaN,NaN,NaN,2.0,NaN,-8442,A,13,CF,A,13,TC
3,Female,43.0,25 - 45,Caucasian,0.0,1.0,0.0,0.0,0.0,0.0,13017627CF10A,1.0,F,Aggravated Battery / Pregnant,0.0,NaN,NS,NaN,NS,0.0,NS,NS,NS,Low,1.0,Low,0.0,1.0,831.0,0.257482,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.0,-15183,0,-1,-15183,NaN,0,NaN,NaN,NaN,-1,0,NaN,NaN,NaN,NaN,2.0,NaN,-15184,A,13,CF,No_Idea,No_Idea,No_Idea
4,Female,25.0,25 - 45,Caucasian,0.0,3.0,0.0,0.0,2.0,-1.0,14028826MU10A,1.0,M,Unlaw Lic Use/Disply Of Others,1.0,15030357TC10A,(M2),NaN,Driving License Suspended,0.0,NS,NS,NS,Low,3.0,Low,2.0,0.0,442.0,1.146142,442.0,-442.0,442.0,-443.0,NaN,443.0,NaN,-1.0,-8553,0,-1,-8554,-8996.0,0,NaN,NaN,NaN,-1,1,NaN,NaN,NaN,NaN,1.0,NaN,-8554,A,14,MU,A,15,TC


In [141]:
train.drop(case_id_vars, axis=1, inplace=True)

In [142]:
train.dtypes.unique()

array([dtype('O'), dtype('float64'), dtype('int64')], dtype=object)

## First model

In [143]:
from catboost import CatBoostClassifier
from catboost import Pool

In [144]:
#train.drop('doughnuts_comsumption', axis=1, inplace=True)

In [145]:
TS=0.1 # Validation size
esr=100 # Early stopping rounds (when validation does not improve in these rounds, stops)

predictoras=list(train)[1:-1]

# Categorical positions for catboost
Categorical=list()
for col in list(train):
    if train[col].dtypes=='object':
        Categorical.append(col)
        
Pos=list()
for col in Categorical:
    Pos.append((train.columns.get_loc(col)))

In [146]:
# To Pool Class (for catboost only)
pool_tr=Pool(train.iloc[index_train, :], y_train,cat_features=Pos)
pool_val=Pool(train.iloc[index_val, :], y_val, cat_features=Pos)

In [169]:
model_catboost = CatBoostClassifier(
		  eval_metric='Accuracy',
		  iterations=10000, # Very high value, to find the optimum
		  od_type='Iter', # Overfitting detector set to "iterations"
		  verbose=100) # Shows train/test metric every "verbose" trees


# "Technical" parameters of the model:
params = {'objective': 'Logloss',
		  'learning_rate': 0.01, # learning rate, lower -> slower but better prediction
		  'depth': 6, # Depth of the trees (values betwwen 5 and 10, higher -> more overfitting)
		  'l2_leaf_reg': 15, # L2 regularization (between 3 and 20, higher -> less overfitting)
		  'rsm': 0.2, # % of features to consider in each split (lower -> faster and reduces overfitting)
		  'subsample': 0.5, # Sample rate for bagging (only if bootstrap_type is Poisson, Bernoulli or MVS)
		  'max_ctr_complexity': 5, # The maximum number of features that can be combined
		  'bootstrap_type': 'Bernoulli'} # Affects important aspects of choosing a split for a tree

''' params calculated by hyperopt
# "Technical" parameters of the model:
params = {'objective': 'Logloss',
		  'learning_rate': 0.01, # learning rate, lower -> slower but better prediction
		  'depth': 5, # Depth of the trees (values betwwen 5 and 10, higher -> more overfitting)
		  'l2_leaf_reg': 14, # L2 regularization (between 3 and 20, higher -> less overfitting)
		  'rsm': 0.17, # % of features to consider in each split (lower -> faster and reduces overfitting)
		  'subsample': 0.58, # Sample rate for bagging (only if bootstrap_type is Poisson, Bernoulli or MVS)
		  'max_ctr_complexity': 7, # The maximum number of features that can be combined
		  'bootstrap_type': 'MVS'} # Affects important aspects of choosing a split for a tree

'''

model_catboost.set_params(**params)

In [168]:
print('\nCatboost Fit (Validation)...\n')
model_catboost.fit(X=pool_tr,
						 eval_set=pool_val,
						 early_stopping_rounds=esr)


Catboost Fit (Validation)...

0:	learn: 0.9876257	test: 0.9882777	best: 0.9882777 (0)	total: 26.5ms	remaining: 4m 25s
100:	learn: 0.9980665	test: 0.9990983	best: 0.9990983 (72)	total: 3.77s	remaining: 6m 9s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.9990982867
bestIteration = 72

Shrink model to first 73 iterations.


In [158]:
importances = model_catboost.feature_importances_

importances = pd.DataFrame([train.columns, importances])
importances = importances.transpose()
importances.columns = ['Variables', 'Importance']
importances = importances.sort_values('Importance', ascending = False)

In [159]:
pd.set_option("display.max_rows", None)

In [160]:
importances

,Variables,Importance
29,c_jail_out_r_offense_date,33.967877
31,c_jail_in_r_offense_date,16.35238
13,is_recid,13.820734
59,r_case_numberletters,7.995093
57,r_case_numberlast,7.126406
58,r_case_numberfirst,6.610662
28,r_offense_date_out_custody,5.380173
27,doughnuts_comsumption,2.415618
30,r_offense_date_screening_date,1.237585
33,r_offense_date_in_custody,0.613967


In [ ]:
# imports for parameter bayesian hyperoptimization
from collections import defaultdict
from hyperopt import hp, tpe, fmin, Trials, space_eval

In [ ]:
class Struct:
    def __init__(self, **entries):
        self.__dict__.update(entries)


# function to minimize by the optimizer (the function returns the -accuracy for each model trained)
def opt_func(space):
    # Params the hyperopt wants to check
	trial_params = Struct(**space)

	model_catboost = CatBoostClassifier(
		eval_metric='Accuracy',
		iterations=10000, # Very high value, to find the optimum
		od_type='Iter', # Overfitting detector set to "iterations"
		verbose=100  # Shows train/test metric every "verbose" trees
	)

	# "Technical" parameters of the model:
	params = {
		'objective': 'Logloss',
		'learning_rate': trial_params.lr, # learning rate, lower -> slower but better prediction
		'depth': trial_params.depth, # Depth of the trees (values betwwen 5 and 10, higher -> more overfitting)
		'l2_leaf_reg': trial_params.l2_leaf_reg, # L2 regularization (between 3 and 20, higher -> less overfitting)
		'rsm': trial_params.rsm, # % of features to consider in each split (lower -> faster and reduces overfitting)
		'subsample': trial_params.subsample, # Sample rate for bagging (only if bootstrap_type is Poisson, Bernoulli or MVS)
		'max_ctr_complexity': trial_params.complexity, # The maximum number of features that can be combined
		'bootstrap_type': trial_params.bootstrap # Affects important aspects of choosing a split for a tree
	} 

	model_catboost.set_params(**params)

    # Train the model with the params decided by the hyperopt
	it_res = model_catboost.fit(X=pool_tr,
						eval_set=pool_val,
						early_stopping_rounds=esr)
	
	return - float(it_res.get_best_score()['learn']['Accuracy'])

In [ ]:
# Define the space where the hyperopt will optimize
trials = Trials()
params = {}
params['lr'] = hp.choice('lr', [0.0001, 0.0005, 0.001, 0.005, 0.01])
params['depth'] = hp.choice('depth', [i for i in range(5, 11)])
params['l2_leaf_reg'] = hp.choice('l2_leaf_reg', [i for i in range(3, 21)])
params['bootstrap'] = hp.choice('bootstrap', ['MVS', 'Bernoulli'])
params['rsm'] = hp.uniform('rsm', 0.1, 0.3)
params['subsample'] = hp.uniform('subsample', 0.2, 0.8)
params['complexity'] = hp.choice('complexity', [3, 4, 5, 6, 7])

space = defaultdict(None, params)

In [ ]:
# optimize
best = fmin(fn=opt_func,
            space=space,
            algo=tpe.suggest,
            max_evals=1000, # limited the hyperopt search, in order to avoid > 800 hour long executions
            trials = trials)

# return parameters that caused the best performance
best